In [15]:
import requests
import pandas as pd
from pyjstat import pyjstat
from collections import OrderedDict
import matplotlib.pyplot as plt
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [16]:
# URL de la API del INEGI
key = '44c77db1-a4bd-c662-9b4c-d22dfbb08213'
url = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/JSONSTAT/6207123177,6207123170,6207123171,6207123163,6207123165,6207123168/es/0700/false/BISE/{key}?type=jsonStat'

# Leer desde json-stat
dataset = pyjstat.Dataset.read(url)

# Escribir a dataframe
df = dataset.write('dataframe')

# Filtrar y renombrar dataframes
df_gt_t = df.iloc[:68, [0, 4, -1]].rename(columns={"ID_TIME": "Periodo", "OBS_VALUE": "Value"}).assign(Indicadores="Gasto Total - Terrestres")
df_gt_a = df.iloc[68:136, [0, 4, -1]].rename(columns={"ID_TIME": "Periodo", "OBS_VALUE": "Value"}).assign(Indicadores="Gasto Total - Aéreos")
df_gm_a = df.iloc[136:204, [0, 4, -1]].rename(columns={"ID_TIME": "Periodo", "OBS_VALUE": "Value"}).assign(Indicadores="Gasto Medio - Aéreos")
df_nv_a = df.iloc[204:272, [0, 4, -1]].rename(columns={"ID_TIME": "Periodo", "OBS_VALUE": "Value"}).assign(Indicadores="Número de Visitantes - Aéreos")
df_nv_t = df.iloc[272:340, [0, 4, -1]].rename(columns={"ID_TIME": "Periodo", "OBS_VALUE": "Value"}).assign(Indicadores="Número de Visitantes - Terrestres")
df_gm_t = df.iloc[340:408, [0, 4, -1]].rename(columns={"ID_TIME": "Periodo", "OBS_VALUE": "Value"}).assign(Indicadores="Gasto Medio - Terrestres")

# Unir todos los dataframes en uno solo
df_final = pd.concat([df_gt_t, df_gt_a, df_gm_a, df_nv_a, df_nv_t, df_gm_t], ignore_index=True)

# Extraer el año de la columna Periodo y crear una nueva columna para el año
df_final['Año'] = df_final['Periodo'].str.extract(r'(\d{4})').astype(int)

In [18]:
# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    html.H1("Dashboard de Turismo"),
    dcc.Dropdown(
        id='indicadores-dropdown',
        options=[
            {'label': 'Gasto Total - Terrestres', 'value': 'Gasto Total - Terrestres'},
            {'label': 'Gasto Total - Aéreos', 'value': 'Gasto Total - Aéreos'},
            {'label': 'Gasto Medio - Aéreos', 'value': 'Gasto Medio - Aéreos'},
            {'label': 'Número de Visitantes - Aéreos', 'value': 'Número de Visitantes - Aéreos'},
            {'label': 'Número de Visitantes - Terrestres', 'value': 'Número de Visitantes - Terrestres'},
            {'label': 'Gasto Medio - Terrestres', 'value': 'Gasto Medio - Terrestres'}
        ],
        value='Gasto Total - Terrestres'
    ),
    dcc.RangeSlider(
        id='rango-anos',
        min=df_final['Año'].min(),
        max=df_final['Año'].max(),
        step=1,
        marks={year: str(year) for year in range(df_final['Año'].min(), df_final['Año'].max() + 1, 1)},
        value=[df_final['Año'].min(), df_final['Año'].max()]
    ),
    dcc.Graph(id='indicadores-graph')
])

# Callback para actualizar el gráfico
@app.callback(
    Output('indicadores-graph', 'figure'),
    [Input('indicadores-dropdown', 'value'),
     Input('rango-anos', 'value')]
)
def update_graph(selected_indicator, selected_years):
    filtered_df = df_final[
        (df_final['Indicadores'] == selected_indicator) &
        (df_final['Año'] >= selected_years[0]) &
        (df_final['Año'] <= selected_years[1])
    ]
    
    fig = {
        'data': [{
            'x': filtered_df['Periodo'],
            'y': filtered_df['value'],
            'type': 'line'
        }],
        'layout': {
            'title': f'{selected_indicator} por Periodo'
        }
    }
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)